In [110]:
import pandas as pd
import yfinance as yf
import datetime
import calendar

## Fetch AAPL stock from yfinance

In [111]:
start_date = "2010-01-01"
end_date = "2024-12-31"
symbol = "AAPL"
stock_data = yf.download(symbol, start = start_date, end = end_date)
stock_data.reset_index(inplace = True)
stock_data.columns = ["date", "close", "high", "low", "open", "volume"]

[*********************100%***********************]  1 of 1 completed


In [112]:
stock_data.head()

,date,close,high,low,open,volume
0,2010-01-04,6.440331,6.455077,6.391278,6.422877,493729600
1,2010-01-05,6.451465,6.487878,6.417459,6.458085,601904800
2,2010-01-06,6.348846,6.477045,6.342225,6.451465,552160000
3,2010-01-07,6.337110,6.379843,6.291067,6.372319,477131200
4,2010-01-08,6.379242,6.379844,6.291369,6.328685,447610800


## Derive month expiry dates: They are usually the third friday of the month

In [113]:
#Iterate through each year from 2011 to 2024 and each month
#derive the first day of the month and then find after how many days the third friday comes
#This would be days to first friday from start of the month + 14 days(for next two fridays)
#eg. Let's say first day of the month is Sunday, next friday would be in 5 days. we add two more weeks ie 14 days.
#We get 5+14 = 19 which is the third friday of the month

third_fridays_of_month_dict = {"date": []}

for i in range(2010, 2025):
    for j in range(1,13):
        first_day_date = datetime.date(i, j, 1)
        first_day_weekday = first_day_date.weekday()
        days_to_friday_from_first_day = (calendar.FRIDAY - first_day_weekday)%7
        third_friday_of_month_date = first_day_date + datetime.timedelta(days=(days_to_friday_from_first_day + 14))
        third_fridays_of_month_dict["date"].append(third_friday_of_month_date)

third_fridays_of_month = pd.DataFrame(third_fridays_of_month_dict)
third_fridays_of_month["date"] = pd.to_datetime(third_fridays_of_month["date"])
third_fridays_of_month.shape
        

(180, 1)

In [114]:
third_fridays_of_month.dtypes

date    datetime64[ns]
dtype: object

In [115]:
#let's find out holidays on the expiry date. If the the third friday of the month doesn't exist in stock_data, then it was holiday.
#We have to go back one day for those dates
merged = third_fridays_of_month.merge(stock_data, on = "date", how = "left", indicator = True)
print(merged[merged["_merge"] == "left_only"])

          date  close  high  low  open  volume     _merge
51  2014-04-18    NaN   NaN  NaN   NaN     NaN  left_only
111 2019-04-19    NaN   NaN  NaN   NaN     NaN  left_only
147 2022-04-15    NaN   NaN  NaN   NaN     NaN  left_only


In [116]:
#So we have 3 days when expiry had a holiday, we reduce 1 day from those dates
holiday_expiry_dates = merged.loc[merged['_merge'] == 'left_only'].copy()
mask = third_fridays_of_month["date"].isin(holiday_expiry_dates["date"])
third_fridays_of_month.loc[mask, "date"] = third_fridays_of_month.loc[mask, "date"] + pd.Timedelta(days = -1)

#Check again
merged = third_fridays_of_month.merge(stock_data, on = "date", how = "left", indicator = True)
print(merged[merged["_merge"] == "left_only"])

Empty DataFrame
Columns: [date, close, high, low, open, volume, _merge]
Index: []


In [117]:
#Add a column to stock_data to identify the expiry day
stock_data['expiry'] = ""
stock_data.loc[stock_data["date"].isin(third_fridays_of_month["date"]), "expiry"] = "expiry_day"
stock_data[stock_data["expiry"] == "expiry_day"].head()

,date,close,high,low,open,volume,expiry
9,2010-01-15,6.197174,6.367805,6.195368,6.347642,594067600,expiry_day
32,2010-02-19,6.068975,6.115019,6.052123,6.074694,415469600,expiry_day
52,2010-03-19,6.688302,6.778281,6.657606,6.764739,559445600,expiry_day
71,2010-04-16,7.445157,7.557707,7.359390,7.480366,750545600,expiry_day
96,2010-05-21,7.292283,7.357887,6.962155,7.006392,1223891200,expiry_day


In [118]:
##Adding first day of the new expiry contract, it will start after the previous expiry date
stock_data.sort_values(by = "date").reset_index(drop = True)
for i in stock_data.index:
    if stock_data.at[i, "expiry"] == "expiry_day":
        stock_data.at[i+1, "expiry"] = "new_day"
stock_data[(stock_data["expiry"] == "expiry_day") | (stock_data["expiry"] == "new_day")].head()

,date,close,high,low,open,volume,expiry
9,2010-01-15,6.197174,6.367805,6.195368,6.347642,594067600,expiry_day
10,2010-01-19,6.471326,6.475840,6.236597,6.269398,730007600,new_day
32,2010-02-19,6.068975,6.115019,6.052123,6.074694,415469600,expiry_day
33,2010-02-22,6.031357,6.093952,5.994343,6.089138,390563600,new_day
52,2010-03-19,6.688302,6.778281,6.657606,6.764739,559445600,expiry_day


## Adding columns for option contract

We first find the strike price interval
At a new day, find a strike price above 5% and above 10% for new_day, that's where we would be taking our trades

In [119]:
stock_data.dtypes

date      datetime64[ns]
close            float64
high             float64
low              float64
open             float64
volume             int64
expiry            object
dtype: object

In [120]:
#Add a new column to find strike price intervals
stock_data = stock_data.round(2)
stock_data["strike_price_intervals"] = ""

for i in stock_data[stock_data["expiry"] == "new_day"].index:
    if stock_data.loc[i, "close"] <= 25.00:
        stock_data.loc[i, "strike_price_intervals"] = 0.50
    elif stock_data.loc[i, "close"] > 25.00 and stock_data.loc[i, "close"] <= 200.00:
        stock_data.loc[i, "strike_price_intervals"] = 1.00
    elif stock_data.loc[i, "close"] > 200.00:
        stock_data.loc[i, "strike_price_intervals"] = 2.5

In [121]:
stock_data[stock_data["expiry"] == "new_day"].head()

,date,close,high,low,open,volume,expiry,strike_price_intervals
10,2010-01-19,6.47,6.48,6.24,6.27,730007600,new_day,0.5
33,2010-02-22,6.03,6.09,5.99,6.09,390563600,new_day,0.5
53,2010-03-22,6.76,6.80,6.63,6.63,456419600,new_day,0.5
72,2010-04-19,7.44,7.46,7.28,7.43,566924400,new_day,0.5
97,2010-05-24,7.43,7.55,7.41,7.44,754238800,new_day,0.5


In [122]:
##Choose strike prices at 5% and 10%
stock_data["sp_5per_away"]  = ""
stock_data["sp_10per_away"] = ""

sp_for_expiry_5per = 0
sp_for_expiry_10per = 0
for i in stock_data[(stock_data["expiry"] == "new_day") | (stock_data["expiry"] == "expiry_day")].index:
    if stock_data.loc[i, "expiry"] == "new_day":
        interval = stock_data.loc[i, "strike_price_intervals"]
        five_per_away = stock_data.loc[i, "open"] * 1.05
        ten_per_away = stock_data.loc[i, "open"] * 1.10
        stock_data.loc[i, "sp_5per_away"] = ((five_per_away//interval) + 1) * interval
        stock_data.loc[i, "sp_10per_away"] = ((ten_per_away//interval) + 1) * interval
        sp_for_expiry_5per = stock_data.loc[i, "sp_5per_away"]
        sp_for_expiry_10per = stock_data.loc[i, "sp_10per_away"]
    else:
        stock_data.loc[i, "sp_5per_away"] = sp_for_expiry_5per
        stock_data.loc[i, "sp_10per_away"] = sp_for_expiry_10per

stock_data[(stock_data["expiry"] == "new_day") | (stock_data["expiry"] == "expiry_day")]

,date,close,high,low,open,volume,expiry,strike_price_intervals,sp_5per_away,sp_10per_away
9,2010-01-15,6.20,6.37,6.20,6.35,594067600,expiry_day,,0,0
10,2010-01-19,6.47,6.48,6.24,6.27,730007600,new_day,0.5,7.0,7.0
32,2010-02-19,6.07,6.12,6.05,6.07,415469600,expiry_day,,7.0,7.0
33,2010-02-22,6.03,6.09,5.99,6.09,390563600,new_day,0.5,6.5,7.0
52,2010-03-19,6.69,6.78,6.66,6.76,559445600,expiry_day,,6.5,7.0
...,...,...,...,...,...,...,...,...,...,...
3724,2024-10-21,235.96,236.33,233.94,233.94,36254500,new_day,2.5,247.5,257.5
3743,2024-11-15,224.75,226.67,224.02,226.15,47923700,expiry_day,,247.5,257.5
3744,2024-11-18,227.77,229.49,224.92,225.00,44686000,new_day,2.5,237.5,250.0
3767,2024-12-20,254.21,254.72,245.42,247.77,147495300,expiry_day,,237.5,250.0


## Merge option prices to the dataframe

Source of option prices: https://www.dolthub.com/repositories/dolthub/options<br>
I extracted the AAPL option chain in csv file<br>
It only has data from 2019, so we will have to get rid of data till 2018

In [123]:
option_chain = pd.read_csv("optionchain.csv")
option_chain.head()

,date,act_symbol,expiration,strike,call_put,bid,ask,vol,delta,gamma,theta,vega,rho
0,2019-02-09,AAPL,2019-02-22,145.0,Call,25.30,26.05,0.4236,0.9773,0.0038,-0.0371,0.0180,0.0540
1,2019-02-09,AAPL,2019-02-22,145.0,Put,0.06,0.08,0.3886,-0.0149,0.0029,-0.0172,0.0125,-0.0010
2,2019-02-09,AAPL,2019-02-22,152.5,Call,17.90,18.55,0.3267,0.9627,0.0075,-0.0419,0.0271,0.0559
3,2019-02-09,AAPL,2019-02-22,152.5,Put,0.12,0.16,0.3166,-0.0330,0.0070,-0.0274,0.0246,-0.0022
4,2019-02-09,AAPL,2019-02-22,157.5,Call,13.30,13.55,0.2950,0.9208,0.0150,-0.0619,0.0492,0.0550


In [124]:
#Keep only call data and Get rid of unnecessary columns
option_chain = option_chain[option_chain["call_put"] == "Call"]
option_chain = option_chain.drop(["act_symbol", "call_put", "ask", "vol", "delta", "gamma", "vega", "rho"], axis = 1)
option_chain.head()

,date,expiration,strike,bid,theta
0,2019-02-09,2019-02-22,145.0,25.30,-0.0371
2,2019-02-09,2019-02-22,152.5,17.90,-0.0419
4,2019-02-09,2019-02-22,157.5,13.30,-0.0619
6,2019-02-09,2019-02-22,162.5,8.75,-0.0850
8,2019-02-09,2019-02-22,167.5,4.80,-0.1099


Only keep new day and expiry day data from stock_data and then merge option_chain with stock_data to premium on that date

In [125]:
#Only keep new day and expiry day data from stock_data and then merge option_chain with stock_data to premium on that date
stock_data = stock_data[(stock_data["expiry"] == "new_day") | (stock_data["expiry"] == "expiry_day")]

#find the smallest new_day date greater than or equal to 2019-02-09(smallest date from option chain)
#delete data before that

temp_min_date = stock_data.loc[(stock_data["date"] >= "2019-02-09") & (stock_data["expiry"] == "new_day"), "date"].min()
stock_data = stock_data[stock_data["date"] >= temp_min_date]
stock_data.head()

,date,close,high,low,open,volume,expiry,strike_price_intervals,sp_5per_away,sp_10per_away
2296,2019-02-19,40.95,41.07,40.60,40.65,75891200,new_day,1.0,43.0,45.0
2314,2019-03-15,44.58,44.87,44.01,44.28,156171600,expiry_day,,43.0,45.0
2315,2019-03-18,45.04,45.13,44.50,44.51,104879200,new_day,1.0,47.0,49.0
2338,2019-04-18,48.83,48.90,48.51,48.66,96783200,expiry_day,,47.0,49.0
2339,2019-04-22,48.99,49.09,48.47,48.59,77758000,new_day,1.0,52.0,54.0


In [126]:
#We merge the stock_data and option_chain on the basis of date, expiry_date, strike price

#getting rid of last row since it is a new day in the dataframe. Last day should be the expiry day for backtesting
stock_data = stock_data.drop(stock_data.tail(1).index)

#Changing column name expiry to isExpiry
stock_data.rename(columns = {"expiry": "isExpiry"}, inplace = True)

#first we need to add the expiry to trade for each new day in stock_data
#we trade in the next isExpiry = expiry_day contract for each each new_day. So we fetch expiry_day dates and add it to the previous new_day
stock_data = stock_data.sort_values(by="date")
stock_data["expiry"] = pd.NaT
stock_data.loc[(stock_data["isExpiry"] == "expiry_day"), "expiry"] = stock_data.loc[(stock_data["isExpiry"] == "expiry_day"), "date"]
stock_data['expiry'] = stock_data['expiry'].bfill()
stock_data

,date,close,high,low,open,volume,isExpiry,strike_price_intervals,sp_5per_away,sp_10per_away,expiry
2296,2019-02-19,40.95,41.07,40.60,40.65,75891200,new_day,1.0,43.0,45.0,2019-03-15
2314,2019-03-15,44.58,44.87,44.01,44.28,156171600,expiry_day,,43.0,45.0,2019-03-15
2315,2019-03-18,45.04,45.13,44.50,44.51,104879200,new_day,1.0,47.0,49.0,2019-04-18
2338,2019-04-18,48.83,48.90,48.51,48.66,96783200,expiry_day,,47.0,49.0,2019-04-18
2339,2019-04-22,48.99,49.09,48.47,48.59,77758000,new_day,1.0,52.0,54.0,2019-05-17
...,...,...,...,...,...,...,...,...,...,...,...
3723,2024-10-18,234.48,235.66,233.50,235.66,46431500,expiry_day,,240.0,250.0,2024-10-18
3724,2024-10-21,235.96,236.33,233.94,233.94,36254500,new_day,2.5,247.5,257.5,2024-11-15
3743,2024-11-15,224.75,226.67,224.02,226.15,47923700,expiry_day,,247.5,257.5,2024-11-15
3744,2024-11-18,227.77,229.49,224.92,225.00,44686000,new_day,2.5,237.5,250.0,2024-12-20
